In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [20]:
subject = 'Pernambuco - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [21]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_PE.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
droped_list = ['Pernambuco - Produção de Cimento (t)',
               'Carteira de Crédito P.Jurídica - Comercial (R$)',
               'Pernambuco - Desemprego',
               'Direcionamento Aplicação (R$)',
               'Air Relative Humidity (%)',
               'Inadimplência - Comercial (%)',
               'Imóvel Área - Privativa (m²)',
               'Carteira de Crédito – Home equity (R$)',
               'Número de Apartamentos',
               'Pernambuco - IDH Longevidade',
               'Valor Imóveis - Compra (R$)']
data = data.drop(droped_list, axis=1)
data

,Ano,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Pernambuco - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Wind Gust (m/s),Pernambuco - Consumo de Cimento (t)
0,2008,8.785657e+07,5.769035e+06,8.834432,7.768216e+07,0.704795,0.660899,5.483335e+08,2.546644e+09,1.874883e+10,...,2.868108,11.760983,0.671922,23.704331,1012.356520,1564.281549,25.868467,21.249870,2.118063,1717.153000
1,2009,8.860216e+07,5.810149e+06,8.858245,7.790384e+07,0.707057,0.665481,5.302764e+08,2.679945e+09,2.055015e+10,...,2.305885,9.472585,0.671804,22.861798,1012.209502,1487.098457,25.873200,21.577129,1.951534,1878.938000
2,2010,9.096526e+07,5.826469e+06,8.909890,7.856512e+07,0.709020,0.669877,5.046425e+08,2.891930e+09,2.342247e+10,...,1.958246,9.332485,0.671411,22.487970,1012.234225,1437.305833,25.750070,21.891970,1.854189,2406.931000
3,2011,9.301827e+07,5.792701e+06,8.937706,7.897343e+07,0.710693,0.674084,4.854622e+08,3.091988e+09,2.604265e+10,...,2.079088,10.985689,0.670748,21.107295,1012.512701,1436.247207,25.736127,21.841067,1.815199,2496.113000
4,2012,9.484266e+07,5.717659e+06,8.949260,7.926901e+07,0.711600,0.677443,4.867661e+08,3.189506e+09,2.708753e+10,...,2.183282,8.159014,0.669292,21.566683,1012.689602,1436.457629,25.711228,21.251350,1.728413,2728.230000
5,2013,9.627386e+07,5.616792e+06,8.939248,7.935398e+07,0.713567,0.682563,5.225848e+08,3.093872e+09,2.523394e+10,...,2.403445,7.913561,0.668984,21.669594,1012.907542,1521.117838,25.685608,20.953182,1.611013,2982.021000
6,2014,9.733647e+07,5.498393e+06,8.923227,7.929548e+07,0.714447,0.686026,5.676413e+08,2.779340e+09,1.954361e+10,...,2.748397,10.350442,0.667649,19.852601,1013.159804,1654.440407,25.780576,20.818540,1.558118,2983.632857
7,2015,9.797155e+07,5.411870e+06,8.896326,7.910750e+07,0.714401,0.688529,5.179392e+08,2.561731e+09,1.327596e+10,...,2.501529,12.454398,0.665120,18.352626,1013.187593,1731.551083,25.909409,21.066299,1.597579,2632.837886
8,2016,9.849009e+07,5.357988e+06,8.887508,7.905537e+07,0.722910,0.701506,4.524144e+08,2.516859e+09,1.011370e+10,...,2.073464,13.111908,0.666277,16.821327,1013.081354,1774.985187,25.897163,21.315125,1.606812,2308.455257
9,2017,9.859294e+07,5.346247e+06,8.880979,7.898921e+07,0.722811,0.702551,4.213689e+08,2.497710e+09,8.456309e+09,...,2.177386,9.479812,0.665302,16.755645,1013.025277,1768.623220,25.854367,21.296659,1.578862,1813.941000


In [22]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Pernambuco - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Wind Gust (m/s)
0,8.785657e+07,5.769035e+06,8.834432,7.768216e+07,0.704795,0.660899,5.483335e+08,2.546644e+09,1.874883e+10,2.351319e+10,...,-7723.574973,2.868108,11.760983,0.671922,23.704331,1012.356520,1564.281549,25.868467,21.249870,2.118063
1,8.860216e+07,5.810149e+06,8.858245,7.790384e+07,0.707057,0.665481,5.302764e+08,2.679945e+09,2.055015e+10,2.446518e+10,...,-8961.781709,2.305885,9.472585,0.671804,22.861798,1012.209502,1487.098457,25.873200,21.577129,1.951534
2,9.096526e+07,5.826469e+06,8.909890,7.856512e+07,0.709020,0.669877,5.046425e+08,2.891930e+09,2.342247e+10,2.606035e+10,...,-9895.682385,1.958246,9.332485,0.671411,22.487970,1012.234225,1437.305833,25.750070,21.891970,1.854189
3,9.301827e+07,5.792701e+06,8.937706,7.897343e+07,0.710693,0.674084,4.854622e+08,3.091988e+09,2.604265e+10,2.845738e+10,...,-9989.170714,2.079088,10.985689,0.670748,21.107295,1012.512701,1436.247207,25.736127,21.841067,1.815199
4,9.484266e+07,5.717659e+06,8.949260,7.926901e+07,0.711600,0.677443,4.867661e+08,3.189506e+09,2.708753e+10,3.181492e+10,...,-9429.326328,2.183282,8.159014,0.669292,21.566683,1012.689602,1436.457629,25.711228,21.251350,1.728413
5,9.627386e+07,5.616792e+06,8.939248,7.935398e+07,0.713567,0.682563,5.225848e+08,3.093872e+09,2.523394e+10,3.629162e+10,...,-7157.441850,2.403445,7.913561,0.668984,21.669594,1012.907542,1521.117838,25.685608,20.953182,1.611013
6,9.733647e+07,5.498393e+06,8.923227,7.929548e+07,0.714447,0.686026,5.676413e+08,2.779340e+09,1.954361e+10,4.143397e+10,...,-3453.461327,2.748397,10.350442,0.667649,19.852601,1013.159804,1654.440407,25.780576,20.818540,1.558118
7,9.797155e+07,5.411870e+06,8.896326,7.910750e+07,0.714401,0.688529,5.179392e+08,2.561731e+09,1.327596e+10,4.469706e+10,...,-1442.919387,2.501529,12.454398,0.665120,18.352626,1013.187593,1731.551083,25.909409,21.066299,1.597579
8,9.849009e+07,5.357988e+06,8.887508,7.905537e+07,0.722910,0.701506,4.524144e+08,2.516859e+09,1.011370e+10,4.640544e+10,...,-3128.377228,2.073464,13.111908,0.666277,16.821327,1013.081354,1774.985187,25.897163,21.315125,1.606812
9,9.859294e+07,5.346247e+06,8.880979,7.898921e+07,0.722811,0.702551,4.213689e+08,2.497710e+09,8.456309e+09,4.607888e+10,...,-5386.337108,2.177386,9.479812,0.665302,16.755645,1013.025277,1768.623220,25.854367,21.296659,1.578862


In [23]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     1878.938000
1     2406.931000
2     2496.113000
3     2728.230000
4     2982.021000
5     2983.632857
6     2632.837886
7     2308.455257
8     1813.941000
9     1645.004000
10    1690.846000
11    2018.834000
12    2022.411000
13    1940.927000
14            NaN
Name: Pernambuco - Consumo de Cimento (t), dtype: float64

In [24]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Pernambuco - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Wind Gust (m/s)
0,-1.894045,1.016308,-1.508819,-2.131804,-1.583249,-1.598164,1.204752,-0.990234,0.166091,-1.487840,...,-0.569731,2.026352,0.984445,1.355095,1.532738,-1.179403,-0.231175,0.290203,-0.299372,1.628734
1,-1.694920,1.229887,-0.890758,-1.702987,-1.220442,-1.290136,0.820657,-0.546688,0.428911,-1.378556,...,-0.838980,0.085558,-0.051710,1.312892,1.177660,-1.613673,-0.847777,0.336507,0.759984,0.803272
2,-1.063798,1.314665,0.449724,-0.423797,-0.905536,-0.994665,0.275392,0.158672,0.847995,-1.195437,...,-1.042057,-1.114490,-0.115145,1.172184,1.020114,-1.540643,-1.245561,-0.868108,1.779145,0.320744
3,-0.515495,1.139248,1.171687,0.366041,-0.637043,-0.711867,-0.132596,0.824346,1.230291,-0.920269,...,-1.062386,-0.697344,0.633403,0.934380,0.438242,-0.718070,-1.254018,-1.004513,1.614369,0.127477
4,-0.028247,0.749420,1.471589,0.937804,-0.491590,-0.486099,-0.104860,1.148829,1.382743,-0.534838,...,-0.940648,-0.337667,-0.646476,0.412716,0.631847,-0.195530,-1.252337,-1.248108,-0.294581,-0.302714
5,0.353986,0.225440,1.211705,1.102177,-0.176084,-0.141927,0.657046,0.830615,1.112297,-0.020933,...,-0.446625,0.422337,-0.757614,0.301975,0.675218,0.448231,-0.576002,-1.498756,-1.259768,-0.884651
6,0.637781,-0.389615,0.795877,0.989019,-0.034819,0.090888,1.615453,-0.215960,0.282052,0.569384,...,0.358807,1.613108,0.345772,-0.176426,-0.090537,1.193372,0.489088,-0.569662,-1.695614,-1.146844
7,0.807393,-0.839081,0.097662,0.625383,-0.042209,0.259082,0.558230,-0.940033,-0.632425,0.943972,...,0.796000,0.760923,1.298413,-1.082994,-0.722687,1.275456,1.105111,0.690753,-0.893603,-0.951240
8,0.945883,-1.118984,-0.131209,0.524538,1.322915,1.131361,-0.835563,-1.089341,-1.093812,1.140087,...,0.429496,-0.716758,1.596124,-0.668215,-1.368038,0.961643,1.452098,0.570944,-0.088137,-0.905473
9,0.973351,-1.179977,-0.300688,0.396558,1.307025,1.201669,-1.495939,-1.153056,-1.335633,1.102599,...,-0.061498,-0.358019,-0.048438,-1.017629,-1.395719,0.796001,1.401274,0.152257,-0.147914,-1.044017


In [25]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     1878.938000
1     2406.931000
2     2496.113000
3     2728.230000
4     2982.021000
5     2983.632857
6     2632.837886
7     2308.455257
8     1813.941000
9     1645.004000
10    1690.846000
11    2018.834000
Name: Pernambuco - Consumo de Cimento (t), dtype: float64

In [26]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Pernambuco - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Wind Gust (m/s)
12,0.243152,-0.752383,-1.854042,-1.166663,0.818285,1.038092,1.108405,2.635823,-0.189959,0.726405,...,0.964765,-1.482779,-2.260447,-1.580713,-1.117166,-0.348766,-0.031355,0.055743,1.110593,0.854739


In [27]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    2022.411
Name: Pernambuco - Consumo de Cimento (t), dtype: float64

In [28]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [29]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
        if winner_seed == 0.0:
            return winner_seed

    return winner_seed

In [30]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3447235554, 3330981641, 2018551586, 1233312446, 1488510008, 1972173115, 2698630663, 1426680204, 2467918818, 1430298237, 2460867858, 4233120633, 801833549, 2535516652, 4194875694, 2037065931, 809246141, 1261546248, 2100295672, 3184396438, 4039386402, 4065272878, 14726786, 2045503598, 1169430779, 3920920446, 404338046, 4245287640, 1728342897, 2354736644, 1733520063, 1986010081, 3038829420, 1121078607, 19684607, 4161015844, 2096973782, 3938033075, 3893253182, 3531184444, 448756426, 851250703, 1264186739, 1597732163, 3885749743, 3818766792, 2847758989, 3493177945, 1099321776, 1351346866, 3608660293, 847282770, 537066875, 242119137, 2410833198, 2533255185, 3234012558, 3782639896, 3635243137, 4020235853, 2751644681, 2921501026, 2445899687, 2737613979, 3224128960, 2634082654, 270187002, 2639394495, 2441920766, 1247374750, 2811137780, 1698026123, 1395314046, 3276669758, 62006273, 3409935373, 3524312291, 63345815, 1770004567, 2167668668, 1119174442, 2696454486, 205668568, 746810037, 3972074381

val_loss: 3275.10302734375


Step: 84 ___________________________________________
val_loss: 5.608201026916504


Step: 85 ___________________________________________
val_loss: 10183.623046875


Step: 86 ___________________________________________
val_loss: 24243.767578125


Step: 87 ___________________________________________
val_loss: 0.03246331214904785


Step: 88 ___________________________________________
val_loss: 436.8926696777344


Step: 89 ___________________________________________
val_loss: 2329.3349609375


Step: 90 ___________________________________________
val_loss: 47992.55078125


Step: 91 ___________________________________________
val_loss: 0.046631112694740295


Step: 92 ___________________________________________
val_loss: 7441.80029296875


Step: 93 ___________________________________________
val_loss: 0.005134478211402893


Step: 94 ___________________________________________
val_loss: 482.67425537109375


Step: 95 ___________________________________________
val_lo

In [31]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 467ms/step - loss: 5499107.5000 - val_loss: 3920221.2500
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 5349655.0000 - val_loss: 1878071.1250
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 4408970.5000 - val_loss: 7393505.5000
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 4611502.5000 - val_loss: 2729636.5000
Epoch 5/10000
1/1 [==============================] - 0s 13ms/step - loss: 5377472.5000 - val_loss: 3167596.0000
Epoch 6/10000
1/1 [==============================] - 0s 13ms/step - loss: 5367663.0000 - val_loss: 5412472.5000
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 4761400.5000 - val_loss: 5150491.5000
Epoch 8/10000
1/1 [==============================] - 0s 15ms/step - loss: 4654109.5000 - val_loss: 4854389.0000
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 4396029.5000 - val_loss: 4948

1/1 [==============================] - 0s 14ms/step - loss: 1413025.0000 - val_loss: 1160880.0000
Epoch 74/10000
1/1 [==============================] - 0s 13ms/step - loss: 1379255.6250 - val_loss: 973031.0000
Epoch 75/10000
1/1 [==============================] - 0s 14ms/step - loss: 1315430.6250 - val_loss: 1168314.5000
Epoch 76/10000
1/1 [==============================] - 0s 14ms/step - loss: 1325140.7500 - val_loss: 1072122.7500
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 1357949.3750 - val_loss: 956360.0625
Epoch 78/10000
1/1 [==============================] - 0s 13ms/step - loss: 1304635.3750 - val_loss: 966968.3125
Epoch 79/10000
1/1 [==============================] - 0s 13ms/step - loss: 1203025.3750 - val_loss: 938927.1875
Epoch 80/10000
1/1 [==============================] - 0s 13ms/step - loss: 1230948.2500 - val_loss: 1015295.9375
Epoch 81/10000
1/1 [==============================] - 0s 13ms/step - loss: 1243145.0000 - val_loss: 946354.6875
Epo

1/1 [==============================] - 0s 13ms/step - loss: 422758.2500 - val_loss: 271179.0312
Epoch 147/10000
1/1 [==============================] - 0s 14ms/step - loss: 422192.7812 - val_loss: 286073.5000
Epoch 148/10000
1/1 [==============================] - 0s 13ms/step - loss: 415152.9062 - val_loss: 279451.3750
Epoch 149/10000
1/1 [==============================] - 0s 13ms/step - loss: 404380.2812 - val_loss: 252799.5469
Epoch 150/10000
1/1 [==============================] - 0s 13ms/step - loss: 401750.2812 - val_loss: 272304.0938
Epoch 151/10000
1/1 [==============================] - 0s 14ms/step - loss: 403179.5938 - val_loss: 241059.7812
Epoch 152/10000
1/1 [==============================] - 0s 13ms/step - loss: 395839.0312 - val_loss: 256177.0938
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 374363.5000 - val_loss: 239974.0469
Epoch 154/10000
1/1 [==============================] - 0s 13ms/step - loss: 380280.8438 - val_loss: 238699.2812
Epoch 15

1/1 [==============================] - 0s 14ms/step - loss: 204652.6250 - val_loss: 68029.7812
Epoch 220/10000
1/1 [==============================] - 0s 13ms/step - loss: 191594.6719 - val_loss: 26550.9766
Epoch 221/10000
1/1 [==============================] - 0s 14ms/step - loss: 211183.8281 - val_loss: 52321.0352
Epoch 222/10000
1/1 [==============================] - 0s 13ms/step - loss: 200945.0469 - val_loss: 48088.8711
Epoch 223/10000
1/1 [==============================] - 0s 14ms/step - loss: 195361.4375 - val_loss: 32794.9453
Epoch 224/10000
1/1 [==============================] - 0s 13ms/step - loss: 203131.3281 - val_loss: 45215.0508
Epoch 225/10000
1/1 [==============================] - 0s 14ms/step - loss: 205488.6250 - val_loss: 42711.1445
Epoch 226/10000
1/1 [==============================] - 0s 14ms/step - loss: 195734.2500 - val_loss: 39986.9141
Epoch 227/10000
1/1 [==============================] - 0s 14ms/step - loss: 196465.9531 - val_loss: 47916.4727
Epoch 228/10000
1

Epoch 293/10000
1/1 [==============================] - 0s 14ms/step - loss: 155594.3594 - val_loss: 15778.3896
Epoch 294/10000
1/1 [==============================] - 0s 13ms/step - loss: 149700.4844 - val_loss: 16432.0664
Epoch 295/10000
1/1 [==============================] - 0s 13ms/step - loss: 150551.9531 - val_loss: 15772.1348
Epoch 296/10000
1/1 [==============================] - 0s 14ms/step - loss: 148044.8594 - val_loss: 15934.2852
Epoch 297/10000
1/1 [==============================] - 0s 13ms/step - loss: 147196.0781 - val_loss: 15749.7900
Epoch 298/10000
1/1 [==============================] - 0s 13ms/step - loss: 147838.7188 - val_loss: 15484.6914
Epoch 299/10000
1/1 [==============================] - 0s 13ms/step - loss: 149622.8906 - val_loss: 14437.5830
Epoch 300/10000
1/1 [==============================] - 0s 13ms/step - loss: 146369.0156 - val_loss: 15836.9258
Epoch 301/10000
1/1 [==============================] - 0s 13ms/step - loss: 147620.2656 - val_loss: 13700.8857
E

...

1/1 [==============================] - 0s 13ms/step - loss: 119217.9766 - val_loss: 839.7332
Epoch 1041/10000
1/1 [==============================] - 0s 14ms/step - loss: 120983.3750 - val_loss: 1383.1190
Epoch 1042/10000
1/1 [==============================] - 0s 14ms/step - loss: 153409.6875 - val_loss: 3908.9512
Epoch 1043/10000
1/1 [==============================] - 0s 13ms/step - loss: 133985.1719 - val_loss: 3074.0203
Epoch 1044/10000
1/1 [==============================] - 0s 13ms/step - loss: 122235.8359 - val_loss: 864.2036
Epoch 1045/10000
1/1 [==============================] - 0s 14ms/step - loss: 154385.8906 - val_loss: 2412.5701
Epoch 1046/10000
1/1 [==============================] - 0s 13ms/step - loss: 124135.5234 - val_loss: 13288.6123
Epoch 1047/10000
1/1 [==============================] - 0s 13ms/step - loss: 134092.1250 - val_loss: 5.5580
Epoch 1048/10000
1/1 [==============================] - 0s 14ms/step - loss: 140460.7656 - val_loss: 856.4268
Epoch 1049/10000
1/1 [=

1/1 [==============================] - 0s 13ms/step - loss: 128043.0625 - val_loss: 523.8801
Epoch 1115/10000
1/1 [==============================] - 0s 13ms/step - loss: 117894.7891 - val_loss: 519.7587
Epoch 1116/10000
1/1 [==============================] - 0s 13ms/step - loss: 138687.2969 - val_loss: 475.6573
Epoch 1117/10000
1/1 [==============================] - 0s 13ms/step - loss: 140241.0000 - val_loss: 55.8479
Epoch 1118/10000
1/1 [==============================] - 0s 13ms/step - loss: 147155.9219 - val_loss: 1510.9050
Epoch 1119/10000
1/1 [==============================] - 0s 14ms/step - loss: 136038.6406 - val_loss: 930.8756
Epoch 1120/10000
1/1 [==============================] - 0s 13ms/step - loss: 124746.6016 - val_loss: 2219.5232
Epoch 1121/10000
1/1 [==============================] - 0s 13ms/step - loss: 131550.8281 - val_loss: 1804.8495
Epoch 1122/10000
1/1 [==============================] - 0s 13ms/step - loss: 134213.4375 - val_loss: 5071.9712
Epoch 1123/10000
1/1 [==

1/1 [==============================] - 0s 13ms/step - loss: 119669.4688 - val_loss: 2837.0579
Epoch 1189/10000
1/1 [==============================] - 0s 14ms/step - loss: 131630.0312 - val_loss: 1904.6959
Epoch 1190/10000
1/1 [==============================] - 0s 14ms/step - loss: 129477.6328 - val_loss: 7849.0254
Epoch 1191/10000
1/1 [==============================] - 0s 13ms/step - loss: 150928.2344 - val_loss: 1078.6959
Epoch 1192/10000
1/1 [==============================] - 0s 14ms/step - loss: 131823.7031 - val_loss: 1174.4677
Epoch 1193/10000
1/1 [==============================] - 0s 14ms/step - loss: 135430.0625 - val_loss: 33.7029
Epoch 1194/10000
1/1 [==============================] - 0s 13ms/step - loss: 127626.2578 - val_loss: 4321.7783
Epoch 1195/10000
1/1 [==============================] - 0s 13ms/step - loss: 121168.7500 - val_loss: 4538.9302
Epoch 1196/10000
1/1 [==============================] - 0s 13ms/step - loss: 125795.4375 - val_loss: 2871.8320
Epoch 1197/10000
1/1

1/1 [==============================] - 0s 13ms/step - loss: 122806.5078 - val_loss: 978.1806
Epoch 1263/10000
1/1 [==============================] - 0s 14ms/step - loss: 141661.7344 - val_loss: 3700.4172
Epoch 1264/10000
1/1 [==============================] - 0s 13ms/step - loss: 139152.6719 - val_loss: 1461.6621
Epoch 1265/10000
1/1 [==============================] - 0s 13ms/step - loss: 133673.0156 - val_loss: 4961.9229
Epoch 1266/10000
1/1 [==============================] - 0s 13ms/step - loss: 113626.7109 - val_loss: 4238.8491
Epoch 1267/10000
1/1 [==============================] - 0s 13ms/step - loss: 107273.3125 - val_loss: 4421.0000
Epoch 1268/10000
1/1 [==============================] - 0s 13ms/step - loss: 132056.1094 - val_loss: 635.7514
Epoch 1269/10000
1/1 [==============================] - 0s 13ms/step - loss: 130321.3750 - val_loss: 4278.3608
Epoch 1270/10000
1/1 [==============================] - 0s 13ms/step - loss: 139560.7031 - val_loss: 1550.3715
Epoch 1271/10000
1/1

1/1 [==============================] - 0s 13ms/step - loss: 132423.3125 - val_loss: 2478.0271
Epoch 1337/10000
1/1 [==============================] - 0s 13ms/step - loss: 140621.0625 - val_loss: 2261.8198
Epoch 1338/10000
1/1 [==============================] - 0s 14ms/step - loss: 139630.6250 - val_loss: 318.6173
Epoch 1339/10000
1/1 [==============================] - 0s 22ms/step - loss: 109170.4609 - val_loss: 10471.2949
Epoch 1339: early stopping


In [32]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 47ms/step
[2022.411](test_target) - [[2018.8271]](prediction) = 3.583851562500058


In [33]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [34]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Pernambuco - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Wind Gust (m/s)
0,-0.969522,-1.359398,-1.050601,-0.981735,-1.252557,-1.233150,1.148205,-1.122043,-1.121418,-1.109924,...,1.277766,1.308949,1.412346,0.959570,1.349075,1.396668,1.302745,0.665154,-1.232563,1.316781
1,-0.406879,0.342020,-0.294564,-0.390692,0.057661,0.016987,0.140893,-0.184474,-0.185492,-0.204018,...,-0.114016,-0.190804,-0.643250,0.419892,-0.307116,-0.890660,-0.174780,0.748246,0.015789,-0.211662
2,1.376400,1.017379,1.345165,1.372427,1.194896,1.216163,-1.289098,1.306518,1.306910,1.313942,...,-1.163750,-1.118144,-0.769096,-1.379462,-1.041959,-0.506008,-1.127966,-1.413400,1.216774,-1.105119


train_target:


,Pernambuco - Consumo de Cimento (t)
0,1878.938
1,2406.931
2,2496.113


test_input:


...

,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Pernambuco - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Wind Gust (m/s)
12,0.243152,-0.752383,-1.854042,-1.166663,0.818285,1.038092,1.108405,2.635823,-0.189959,0.726405,...,0.964765,-1.482779,-2.260447,-1.580713,-1.117166,-0.348766,-0.031355,0.055743,1.110593,0.854739


test_target:


,Pernambuco - Consumo de Cimento (t)
12,2022.411


1/1 [==============================] - 0s 34ms/step
Error: 3.583851562500058




[2496.09521484375,
 2623.486083984375,
 3241.08642578125,
 2976.13623046875,
 2525.3828125,
 2308.7275390625,
 2216.96337890625,
 1871.673583984375,
 1991.914794921875,
 2018.8271484375]

In [35]:
display(targets)
display(predictions)

[2728.23,
 2982.021,
 2983.632857142857,
 2632.8378857142857,
 2308.455257142857,
 1813.941,
 1645.004,
 1690.846,
 2018.834,
 2022.411]

[2496.09521484375,
 2623.486083984375,
 3241.08642578125,
 2976.13623046875,
 2525.3828125,
 2308.7275390625,
 2216.96337890625,
 1871.673583984375,
 1991.914794921875,
 2018.8271484375]

In [36]:
mae = mean_absolute_error(predictions, targets)
mae

268.64257285156253

In [37]:
porcentage = mae/np.mean(targets)
porcentage

0.11769038204084162